In [4]:
# 여러게시물 #상세내용 #수집 웹 크롤러 
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select


#1. 크롤링 기본 셋팅
print("=" *80)
print("연습문제 6-3 : 청와대 국민 신문고 게시판 크롤링하기")
print("=" *80)
query_txt = '청와대국민신문고'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)


f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): ")
if f_dir=='' :
    f_dir='/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

    
# 저장될 파일위치와 이름 지정하기
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name= s+'-'+query_txt+'.txt'
fc_name= s+'-'+query_txt+'.csv'
fx_name= s+'-'+query_txt+'.xls'


# 크롬 드라이버를 사용해서 웹 브라우저 실행하기 

s_time = time.time( )

path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)

#1페이지로 이동
driver.get('https://www.epeople.go.kr/nep/prpsl/opnPrpl/opnpblPrpslList.npaid')
time.sleep(random.randrange(2,5))


html1 = driver.page_source 
soup1 = BeautifulSoup(html1, 'html.parser')
total_number = soup1.find('div', 'prog_util').find('span', 'total').find('span').get_text()
total_number1 = int(total_number.replace(',', "").strip())

#검색건수 조정하기
if cnt > total_number1 : 
    cnt = total_number1 
    print('='*100)
    print('실제 요청하신 검색 건수는 %s 건 이었지만' %cnt)
    print('총 검색된 건수는 %s 건 이어서' %total_number1)
    print('총 %s건만 출력하겠습니다' %total_number1)
    print('='*100)
    page_cnt = math.ceil(total_number1/10)


#내용 추출하기 
pet_number1 = []
pet_title1 = []
pet_op1 = []
pet_da1 = []
pet_sit1 = []
pet_find1 = []

no1 = 1

for a in range(1, page_cnt+1) : 
    html0 = driver.page_source 
    soup0 = BeautifulSoup(html0, 'html.parser')
    list_1 = soup0.find('table', 'tbl default brd1').find('tbody')
    
    f = open(ff_name, 'a', encoding='UTF-8')
    
    
    #페이지 안에 있는 모든 tr 수집
    tr_list = list_1.find_all('tr')
    for b in tr_list : 
        f.write('%s 번째 국민 신문고 게시글 정보입니다=============================' %no1)
        f.write('\n')

        td_list = b.find_all('td')
        
        petition_number = td_list[0].get_text().strip()
        f.write('1. 청원번호:'+petition_number)
        f.write('\n')
        pet_number1.append(petition_number)
        
        petition_title = td_list[1].get_text().replace('\n', "").strip()
        f.write('2. 청원제목:'+petition_title)
        f.write('\n')
        pet_title1.append(petition_title)
        
        petition_operation = td_list[2].get_text().strip()
        f.write('3. 처리기관명:'+petition_operation)
        f.write('\n')
        pet_op1.append(petition_operation)
        
        petition_date = td_list[3].get_text().strip()
        f.write('4. 신청일:'+petition_date)
        f.write('\n')
        pet_da1.append(petition_date)
        
        petition_sit = td_list[4].get_text().strip()
        f.write('5. 추진상황:'+petition_sit)
        f.write('\n')
        pet_sit1.append(petition_sit)
        
        petition_find = td_list[5].get_text().strip()
        f.write('6. 조회수:'+petition_find)
        f.write('\n')
        f.write('\n')
        pet_find1.append(petition_find)
        
        no1 +=1
        
        if no1 > cnt : 
            break
            
    a+1
    if a > page_cnt : 
        break 
    
    
    #다음페이지로 넘어가기
    try : 
        element1 = Select(driver.find_element_by_class_name('page_list'))
        element1.select_by_onclick("frmPageLink(%s);return false; " %a)
        time.sleep(2)
    except : 
        driver.find_element_by_xpath('/html/body/div[3]/main/div/section/article/form[2]/div[3]/span[4]/a/img').click()
        time.sleep(2)  
        
f.close()#txt파일 닫고
driver.close()
        
e_time = time.time()
total_time = e_time - s_time
    

#csv, xlsx파일로 결과물 저장하기 

petition = pd.DataFrame()
petition['글번호'] = pet_number1
petition['글제목'] = pet_title1
petition['처리기관명'] = pet_op1
petition['신청일'] = pet_da1
petition['추진상황'] = pet_sit1
petition['조회수'] = pet_find1

petition.to_csv(fc_name, encoding='utf-8-sig', index=False)
petition.to_excel(fx_name, index=False)

print('='*100)
print('데이터 크롤링에 걸린 총 시간은 %s 이며' %total_time)
print('크롤링된 데이터는 총 %s 건 이고' %cnt)
print('txt, csv, xlsx 파일로 입력하신 경로에 파일 저장하였습니다')
print('='*100)


연습문제 6-3 : 청와대 국민 신문고 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 100
2.결과 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): 
데이터 크롤링에 걸린 총 시간은 40.42784786224365 이며
크롤링된 데이터는 총 100 건 이고
txt, csv, xlsx 파일로 입력하신 경로에 파일 저장하였습니다


In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/a[2]

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/a[2]

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/a[3]

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/span[4]/a/img

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/span[5]/a/img

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/span[4]/a/img

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/span[5]/a/img

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/span[2]/a/img

In [ ]:
/html/body/div[3]/main/div/section/article/form[2]/div[3]/span[1]/a/img